<a href="https://colab.research.google.com/github/dudaholandah/PIBIC/blob/main/vegan_dataset_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
!apt-get install poppler-utils 
!pip install pdf2image
!pip install unidecode
!pip install umap-learn[plot]
!pip install trimap
!pip install -U kaleido
!pip install scikit-learn-extra

In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode
from sklearn import preprocessing
from pandas.core.frame import DataFrame
import plotly.express as px
from sklearn.manifold import TSNE
import re
import umap
import trimap
from sklearn.decomposition import PCA as sklearnPCA
from sklearn.manifold import Isomap
from sklearn.manifold import trustworthiness
import plotly.graph_objects as go
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
import plotly
import pdf2image
import os
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn_extra.cluster import KMedoids

# Pre-Processing Data

In [3]:
data = pd.read_excel("vegan_dataset.xlsx", sheet_name='Veganos')

In [4]:
data.head()

,Classification,Description,Ingredients,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,MEAT 2,Carne Moída do Futuro,"water, texturizedsoyprotein, peaprotein e chic...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,MEAT 3,Carne Moída do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,POULTRY 3,Frango do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",80.0,168.0,11.0,0.0,8.2,10.0,9.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,PORK 1,Linguiça do Futuro,"water, texturizedsoyprotein, isolatedsoyprotei...",50.0,139.0,7.7,0.0,5.7,9.5,3.9,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,MEAT 1,Hamburguer de Soja Goshen,"texturizedsoyprotein, water, soyoil, modified...",50.0,118.0,2.8,0.0,7.0,9.0,1.3,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Separating data

In [5]:
data_ingredients = pd.DataFrame(data['Ingredients'])
data_nutrients = data.drop(columns=['Ingredients', 'Classification', "Description"])
data_clasification = pd.DataFrame(data['Classification'])
data_name = pd.DataFrame(data["Description"])
label_data = data_clasification.join(data_name)

In [6]:
data_nutrients.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
1,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
2,80.0,168.0,11.0,0.0,8.2,10.0,9.0,0.7,392.0,0.0,...,13.75,0.0,10.25,12.5,11.25,0.875,490.0,0.0,0.0,0.0
3,50.0,139.0,7.7,0.0,5.7,9.5,3.9,0.2,294.0,0.0,...,15.40,0.0,11.40,19.0,7.80,0.400,588.0,0.0,0.0,0.0
4,50.0,118.0,2.8,0.0,7.0,9.0,1.3,2.0,344.0,0.0,...,5.60,0.0,14.00,18.0,2.60,4.000,688.0,0.0,0.0,0.0


### Normalizing Nutrient Values (MinMax Scaler) 

In [7]:
X = data_nutrients.values
print(X)

[[ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 [ 80.  168.   11.  ...   0.    0.    0. ]
 ...
 [ 40.   84.    5.2 ...   0.    0.    0. ]
 [ 10.   25.    3.3 ...   0.    0.    0. ]
 [  7.   28.    1.2 ...   0.    0.    0. ]]


In [8]:
attributes_dummies = data_nutrients.columns
normalize = preprocessing.MinMaxScaler()
xscaled = normalize.fit_transform(X)

nutrients_normalized = pd.DataFrame(xscaled,columns=attributes_dummies)
nutrients_normalized = nutrients_normalized.replace(np.nan,0)

nutrients_normalized.head()

,Serving Size,Kcal,Carbohydrate,Sugars,Proteins,Total Fats,Saturated Fats,Dietary Fiber,Sodium,B12,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,0.212828,0.263815,0.244444,0.0,0.195238,0.204082,0.489130,0.007447,0.414376,0.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,0.125364,0.212121,0.171111,0.0,0.135714,0.193878,0.211957,0.002128,0.310782,0.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,0.125364,0.174688,0.062222,0.0,0.166667,0.183673,0.070652,0.021277,0.363636,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


### Normalizing Ingredients (One Hot Encode)

In [9]:
ingredients_normalized = pd.DataFrame()

for i in range(276):
  aux = data_ingredients.at[i,'Ingredients'].split(",")
  for w in aux:
    w = unidecode(w.lower().lstrip().replace('.',''))
    ingredients_normalized.at[i,w] = 1

ingredients_normalized = ingredients_normalized.replace(np.nan,0)
ingredients_normalized.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,cloreto de potassio,calciumdisodium and citricacid,isolatedvegetableprotein,hydrolyzedvegetableprotein,soy,potassiumsorbate,peastarch,peaprotein and sodiumbicarbonate,isolatedriceprotein,chickpeas and goldenflaxseed
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Preparing data

In [10]:
raw_data = pd.DataFrame(ingredients_normalized)
raw_data = pd.concat([raw_data,nutrients_normalized], axis=1)
raw_data = raw_data.replace(np.nan,0)

raw_data.head()

,water,texturizedsoyprotein,peaprotein e chickpeaflour,vegetalfat,modifiedstarch,onion,meatflavoredcondiment,salt,sugar,powderedbeet,...,Carbohydrate.1,Sugars.1,Proteins.1,Total Fats,Saturated Fats,Dietary Fiber .1,Sodium.1,B12.1,Calcium.1,Zinc.1
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.1650,0.0,0.205,0.051020,0.383523,0.001117,0.377892,0.0,0.0,0.0
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.1848,0.0,0.228,0.077551,0.265909,0.000511,0.453470,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0672,0.0,0.280,0.073469,0.088636,0.005106,0.530591,0.0,0.0,0.0


In [11]:
X = raw_data.values
X_ingredients = ingredients_normalized.values
X_nutrients = nutrients_normalized.values
y = label_data.values
all_data = label_data.join(raw_data)

for i in range(276):
  label = label_data['Classification'][i].rstrip()
  label_data.at[i,'Classification'] = label
  all_data.at[i,'Classification'] = label

print(all_data['Classification'].value_counts())

DAIRY 1      80
DAIRY 3      45
MEAT 1       34
MEAT 3       18
POULTRY 1    14
EGG 1        14
POULTRY 3    13
PORK 1       12
DAIRY 2      11
MEAT 2        9
FISH 1        8
PORK 2        6
POULTRY 2     5
FISH 2        5
EGG 2         2
Name: Classification, dtype: int64


# Visualization Techniques

In [12]:
for i in range(276):
  aux = data_ingredients.at[i,'Ingredients']
  data_ingredients.at[i,'Ingredients'] = re.sub(',\s*',"<br>",aux).lstrip()

In [13]:
if not os.path.exists("imgs"):
    os.mkdir("imgs")

### t-SNE

In [14]:
tsne = TSNE(n_components=2,perplexity=5,learning_rate=100,metric='euclidean', init='pca')
X_tsne = tsne.fit_transform(X)

X_tsne[1:4, :]
all_data['x'] = X_tsne[:,0]
all_data['y'] = X_tsne[:,1]
all_data['Ingredients'] = data_ingredients

fig = px.scatter(
    all_data, 
    x='x', 
    y='y', 
    color='Classification', 
    template="simple_white",
    labels={ 
      "Classification": "Classification (Label)"
    },
    color_discrete_sequence= px.colors.qualitative.Plotly + px.colors.qualitative.Bold,
    hover_name="Description",
    hover_data={'x':False,
                'y':False,  
                'Kcal ': [f' {x}' for x in data['Kcal']],
                'Serving Size ': [f' {y}' for y in data['Serving Size']],
                'Carbohydrate ': [f' {z}' for z in data['Carbohydrate']],
                'Sugars ': [f' {a}' for a in data['Sugars']],
                'Proteins ': [f' {b}' for b in data['Proteins']],
                }, #'Ingredients':True}, 
    width=700)

fig.update_traces(showlegend=False)

fig.update_layout(xaxis={'visible': False},
                  yaxis={'visible': False},
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/TSNE_ALL.eps', format='eps')

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


### UMAP

In [15]:
reducer = umap.UMAP(n_components=2,n_neighbors=10,metric='euclidean', random_state=0)
X_umap = reducer.fit_transform(X)

X_umap[1:4, :]
all_data['x'] = X_umap[:,0]
all_data['y'] = X_umap[:,1]

fig = px.scatter(
    all_data, 
    x='x', 
    y='y', 
    color='Classification', 
    template="simple_white",
    labels={ 
      "Classification": "Classification (Label)"
    },
    color_discrete_sequence= px.colors.qualitative.Plotly + px.colors.qualitative.Bold,
    hover_name="Description",
    hover_data={'x':False,
                'y':False,  
                'Kcal ': [f' {x}' for x in data['Kcal']],
                'Serving Size ': [f' {y}' for y in data['Serving Size']],
                'Carbohydrate ': [f' {z}' for z in data['Carbohydrate']],
                'Sugars ': [f' {a}' for a in data['Sugars']],
                'Proteins ': [f' {b}' for b in data['Proteins']],
                }, #'Ingredients':True}, 
    width=700)

fig.update_traces(showlegend=False)

fig.update_layout(xaxis={'visible': False},
                  yaxis={'visible': False},
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/UMAP_ALL.eps', format='eps')

### PCA

In [16]:
sklearn_pca = sklearnPCA(n_components=2)
X_pca = sklearn_pca.fit_transform(X)

X_pca[1:4, :]
all_data['x'] = X_pca[:,0]
all_data['y'] = X_pca[:,1]

fig = px.scatter(
    all_data, 
    x='x', 
    y='y', 
    color='Classification', 
    template="simple_white",
    labels={ 
      "Classification": "Classification (Label)"
    },
    color_discrete_sequence= px.colors.qualitative.Plotly + px.colors.qualitative.Bold,
    hover_name="Description",
    hover_data={'x':False,
                'y':False,  
                'Kcal ': [f' {x}' for x in data['Kcal']],
                'Serving Size ': [f' {y}' for y in data['Serving Size']],
                'Carbohydrate ': [f' {z}' for z in data['Carbohydrate']],
                'Sugars ': [f' {a}' for a in data['Sugars']],
                'Proteins ': [f' {b}' for b in data['Proteins']],
                }, #'Ingredients':True}, 
    width=700)

fig.update_traces(showlegend=False)

fig.update_layout(xaxis={'visible': False},
                  yaxis={'visible': False},
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/PCA_ALL.eps', format='eps')

### TRIMAP

In [17]:
X_trimap = trimap.TRIMAP(n_dims=2, n_inliers=15).fit_transform(X)

X_trimap[1:4, :]
all_data['x'] = X_trimap[:,0]
all_data['y'] = X_trimap[:,1]
all_data['Ingredients'] = data_ingredients

fig = px.scatter(
    all_data, 
    x='x', 
    y='y', 
    color='Classification', 
    template="simple_white",
    labels={ 
      "Classification": "Classification (Label)"
    },
    color_discrete_sequence= px.colors.qualitative.Plotly + px.colors.qualitative.Bold,
    hover_name="Description",
    hover_data={'x':False,
                'y':False,  
                'Kcal ': [f' {x}' for x in data['Kcal']],
                'Serving Size ': [f' {y}' for y in data['Serving Size']],
                'Carbohydrate ': [f' {z}' for z in data['Carbohydrate']],
                'Sugars ': [f' {a}' for a in data['Sugars']],
                'Proteins ': [f' {b}' for b in data['Proteins']],
                #'Ingredients':True},
                }, 
    width=800)

fig.update_layout(xaxis={'visible': False},
                  yaxis={'visible': False},
                  legend=dict(font=dict(size=16)),
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/TRIMAP_ALL.eps', format='eps')

### Not used in the Article

In [18]:
data_ingredients = label_data.join(ingredients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsneI = tsne.fit_transform(X_ingredients)

X_tsneI[1:4, :]
data_ingredients['x'] = X_tsneI[:,0]
data_ingredients['y'] = X_tsneI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [19]:
data_nutrients = label_data.join(nutrients_normalized)
tsne = TSNE(n_components=2,perplexity=5,learning_rate=350.0,metric='euclidean', init='random')
X_tsneN = tsne.fit_transform(nutrients_normalized.values)

X_tsneN[1:4, :]
data_nutrients['x'] = X_tsneN[:,0]
data_nutrients['y'] = X_tsneN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [20]:
data_ingredients = label_data.join(ingredients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=80,metric='euclidean')
X_umapI = reducer.fit_transform(X_ingredients)

X_umapI[1:4, :]
data_ingredients['x'] = X_umapI[:,0]
data_ingredients['y'] = X_umapI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [21]:
data_nutrients = label_data.join(nutrients_normalized)
reducer = umap.UMAP(n_components=2,n_neighbors=80,metric='euclidean')
X_umapN = reducer.fit_transform(X_nutrients)

X_umapN[1:4, :]
data_nutrients['x'] = X_umapN[:,0]
data_nutrients['y'] = X_umapN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [22]:
data_ingredients = label_data.join(ingredients_normalized)
sklearn_pca = sklearnPCA(n_components=200)
X_pcaI = sklearn_pca.fit_transform(X_ingredients)

X_pcaI[1:4, :]
data_ingredients['x'] = X_pcaI[:,0]
data_ingredients['y'] = X_pcaI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [23]:
data_nutrients = label_data.join(nutrients_normalized)
sklearn_pca = sklearnPCA(n_components=20)
X_pcaN = sklearn_pca.fit_transform(X_nutrients)

X_pcaN[1:4, :]
data_nutrients['x'] = X_pcaN[:,0]
data_nutrients['y'] = X_pcaN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [24]:
data_ingredients = label_data.join(ingredients_normalized)
isomap = Isomap(n_components=2,n_neighbors=90)
X_isomapI = isomap.fit_transform(X_ingredients)

X_isomapI[1:4, :]
data_ingredients['x'] = X_isomapI[:,0]
data_ingredients['y'] = X_isomapI[:,1]

fig = px.scatter(data_ingredients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

In [25]:
data_nutrients = label_data.join(nutrients_normalized)
isomap = Isomap(n_components=2,n_neighbors=90)
X_isomapN = isomap.fit_transform(X_nutrients)

X_isomapN[1:4, :]
data_nutrients['x'] = X_isomapN[:,0]
data_nutrients['y'] = X_isomapN[:,1]

fig = px.scatter(data_nutrients, x='x', y='y', color='Classification', hover_data=["Description"], width=800)
fig.show()

# Trustworthiness

## Neighborhood Preservation

### Trustworthiness

All Data (Ingredients + Nutrients)

In [26]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_trimap = []

precision = trustworthiness(X, X_tsne, n_neighbors=5, metric='euclidean')
print(precision)

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsne, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umap, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pca, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_trimap, n_neighbors=k, metric='euclidean')
  y_trimap.append(precision)

0.893191650443435


In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE',
                    line=dict(color=px.colors.qualitative.Plotly[0])))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP',
                    line=dict(color=px.colors.qualitative.Plotly[1])))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA',
                    line=dict(color=px.colors.qualitative.Plotly[2])))

fig.add_trace(go.Scatter(x=kneigh, y=y_trimap,
                    mode='lines+markers',
                    name='TRIMAP',
                    line=dict(color=px.colors.qualitative.Plotly[3])))

fig.update_layout(width=1000, template="simple_white",
                  xaxis_title="K (number of neighbors)",
                  yaxis_title="Trustworthiness",
                  font=dict(size=18),
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/neighborhoodpres.eps', format='eps')

Only Ingredients

In [28]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsneI, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umapI, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pcaI, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomapI, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [29]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

Only Nutrients

In [30]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_isomap = []

for k in range(1,51):
  kneigh.append(k)

  precision = trustworthiness(X, X_tsneN, n_neighbors=k, metric='euclidean')
  y_tsne.append(precision)

  precision = trustworthiness(X, X_umapN, n_neighbors=k, metric='euclidean')
  y_umap.append(precision)

  precision = trustworthiness(X, X_pcaN, n_neighbors=k, metric='euclidean')
  y_pca.append(precision)

  precision = trustworthiness(X, X_isomapN, n_neighbors=k, metric='euclidean')
  y_isomap.append(precision)

In [31]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE'))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP'))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA'))

fig.add_trace(go.Scatter(x=kneigh, y=y_isomap,
                    mode='lines+markers',
                    name='Isomap'))

fig.show()

### KNN

In [32]:
def neighborhood_preservation(lista_a, lista_b, k):
  ans = []
  size = len(lista_a)

  for i in range(size):
    prop = 0.0
    for x in lista_b[i]:
      if x in lista_a[i]: prop += 1
    
    ans.append(prop/k)

  return sum(ans)/size

In [33]:
def knn(X, k):
  nearest_neighbors_ids = []
    
  for p in range(len(X)):
    distances = np.linalg.norm(X - X[p], axis=1)
    nearest_neighbors_ids.append(distances.argsort()[:(k)])

  return nearest_neighbors_ids

Neighborhood Preservation

In [34]:
kneigh = []
y_tsne = []
y_umap = []
y_pca = []
y_trimap = []

for k in range(1,51):
  kneigh.append(k)
  knn_x = knn(X, k)

  knn_tsne = knn(X_tsne, k)
  precision = neighborhood_preservation(knn_x, knn_tsne, k)
  y_tsne.append(precision)

  knn_umap = knn(X_umap, k)
  precision = neighborhood_preservation(knn_x, knn_umap, k)
  y_umap.append(precision)

  knn_pca = knn(X_pca, k)
  precision = neighborhood_preservation(knn_x, knn_pca, k)
  y_pca.append(precision)

  knn_trimap = knn(X_trimap, k)
  precision = neighborhood_preservation(knn_x, knn_trimap, k) 
  y_trimap.append(precision)  

In [35]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_tsne,
                    mode='lines+markers',
                    name='t-SNE',
                    line=dict(color=px.colors.qualitative.Plotly[0])))

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                    mode='lines+markers',
                    name='UMAP',
                    line=dict(color=px.colors.qualitative.Plotly[1])))

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                    mode='lines+markers',
                    name='PCA',
                    line=dict(color=px.colors.qualitative.Plotly[2])))

fig.add_trace(go.Scatter(x=kneigh, y=y_trimap,
                    mode='lines+markers',
                    name='TRIMAP',
                    line=dict(color=px.colors.qualitative.Plotly[3])))

fig.update_layout(width=1000, template="simple_white",
                  xaxis_title="K (number of neighbors)",
                  yaxis_title="Neighborhood Preservation",
                  font=dict(size=18),
                  margin=dict(l=0,r=0,b=0,t=0))

fig.show()
fig.write_image('imgs/neighborhoodpres.eps', format='eps')

Neighborhood Hit

# Silhouette Coefficient

In [36]:
def coeficiente_silhueta(X_high,X_low):
  silhouette_avg_high = []
  silhouette_avg_low = []

  for k in range(3,20):
    clusterer = KMedoids(n_clusters=k, init='k-medoids++',random_state=10)
    cluster_labels_high = clusterer.fit_predict(X_high)
    cluster_labels_low = clusterer.fit_predict(X_low)
    silhouette_avg_high.append(silhouette_score(X_high, cluster_labels_high))
    silhouette_avg_low.append(silhouette_score(X_low, cluster_labels_low))

  mean_silhouete_high = np.mean(np.array(silhouette_avg_high))
  mean_silhouete_low = np.mean(np.array(silhouette_avg_low))
  std_silhouete_high = np.std(np.array(silhouette_avg_high))
  std_silhouete_low = np.std(np.array(silhouette_avg_low))
    
  return mean_silhouete_high, mean_silhouete_low, std_silhouete_high, std_silhouete_low

### t-SNE

In [37]:
mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_tsne)

print("TSNE")
print(f"Média Silhueta Original: {mean_high} Desvio Padrão Original: {std_high}")
print(f"Média Silhueta Visualização: {mean_low:.10f} Desvio Padrão Visualização: {std_low:.10f}\n")

TSNE
Média Silhueta Original: 0.056157551079033335 Desvio Padrão Original: 0.019028980449068117
Média Silhueta Visualização: 0.3916836679 Desvio Padrão Visualização: 0.0361868739



### UMAP

In [38]:
mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_umap)

print("UMAP")
print(f"Média Silhueta Original: {mean_high} Desvio Padrão Original: {std_high}")
print(f"Média Silhueta Visualização: {mean_low:.10f} Desvio Padrão Visualização: {std_low:.10f}\n")

UMAP
Média Silhueta Original: 0.056157551079033335 Desvio Padrão Original: 0.019028980449068117
Média Silhueta Visualização: 0.4858280122 Desvio Padrão Visualização: 0.0267988946



### PCA

In [39]:
mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_pca)

print("PCA")
print(f"Média Silhueta Original: {mean_high} Desvio Padrão Original: {std_high}")
print(f"Média Silhueta Visualização: {mean_low:.10f} Desvio Padrão Visualização: {std_low:.10f}\n")

PCA
Média Silhueta Original: 0.056157551079033335 Desvio Padrão Original: 0.019028980449068117
Média Silhueta Visualização: 0.4002078598 Desvio Padrão Visualização: 0.0424993462



### TRIMAP

In [40]:
mean_high,mean_low,std_high,std_low = coeficiente_silhueta(X,X_trimap)

print("TriMap")
print(f"Média Silhueta Original: {mean_high} Desvio Padrão Original: {std_high}")
print(f"Média Silhueta Visualização: {mean_low:.10f} Desvio Padrão Visualização: {std_low:.10f}\n")

TriMap
Média Silhueta Original: 0.056157551079033335 Desvio Padrão Original: 0.019028980449068117
Média Silhueta Visualização: 0.5344805717 Desvio Padrão Visualização: 0.0351237953



# Global Score


### t-SNE

In [41]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_tsne)
print(f"tSNE global score {gs} - All Data")

tSNE global score 0.9520419412120931 - All Data


### UMAP

In [42]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_umap)
print(f"UMAP global score {gs} - All Data")

UMAP global score 0.9367252251214411 - All Data


### PCA

In [43]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_pca)
print(f"PCA global score {gs} - All Data")

PCA global score 0.999999999106895 - All Data


### TRIMAP

In [44]:
gs = trimap.TRIMAP(verbose=False).global_score(X, X_trimap)
print(f"TRIMAP global score {gs} - All Data")

TRIMAP global score 0.9606665885762683 - All Data


# Testing values

### t-SNE

In [ ]:
kneigh = []
y_tsne = np.zeros((10,21))

v_perplexity = [5,10,15,20,25,30,35,40,45,50]
v_learning_rate = [10,50,100,150,200,250,300,350,400,450,500,550,600,650,700,750,800,850,900,950,1000]

for pv in range(0,21):
  kneigh.append(v_learning_rate[pv])
  for pi in range(0,10):
    tsne = TSNE(n_components=2,perplexity=v_perplexity[pi],learning_rate=v_learning_rate[pv],metric='euclidean', init='pca')
    X_tsne = tsne.fit_transform(X)
    precision = trustworthiness(X, X_tsne, metric='euclidean')
    y_tsne[pi][pv] = precision

In [ ]:
fig = go.Figure()
for pi in range(10):
  fig.add_trace(go.Scatter(x=kneigh, y=y_tsne[pi],
                    mode='lines+markers',
                    name='t-SNE perplexity ' + str(v_perplexity[pi])))

fig.show()

### UMAP

In [ ]:
teste = [0,0.1,0.25,0.5,0.8,0.99]
kneigh = []
y_umap = []

for n in teste:
  kneigh.append(n)
  reducer = umap.UMAP(n_components=2, n_neighbors=10, min_dist=n, metric='euclidean')
  X_umap = reducer.fit_transform(X)
  precision = trustworthiness(X, X_umap, metric='euclidean')
  y_umap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

In [ ]:
teste = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_umap = []

for n in teste:
  kneigh.append(n)
  reducer = umap.UMAP(n_components=n,n_neighbors=40,metric='euclidean')
  X_umap = reducer.fit_transform(X)
  precision = trustworthiness(X, X_umap, n_neighbors=1, metric='euclidean')
  y_umap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_umap,
                  mode='lines+markers',
                  name='UMAP'))

fig.show()

### PCA

In [ ]:
components = [2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
kneigh = []
y_pca = []

for n in components:
  kneigh.append(n)
  pca = sklearnPCA(n_components=n)
  X_pca = pca.fit_transform(X)
  precision = trustworthiness(X, X_pca, n_neighbors=1, metric='euclidean')
  y_pca.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_pca,
                  mode='lines+markers',
                  name='PCA'))

fig.show()

### TRIMAP

In [ ]:
teste = [1,2,3,4,5,6,7,8,9,10]
kneigh = []
y_trimap = []

for n in teste:
  kneigh.append(n)
  X_trimap = trimap.TRIMAP(n_dims=2, n_inliers=5, n_random=n).fit_transform(X)
  precision = trustworthiness(X, X_trimap, metric='euclidean')
  y_trimap.append(precision)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=kneigh, y=y_trimap,
                  mode='lines+markers',
                  name='TRIMAP'))

fig.show()